# **Building an RAG Query pipeline with FAISS, and Ollama’s Llama 2 Model**

This project develops a Retrieval-Augmented Generation (RAG) query app that combines efficient retrieval with AI-driven responses. By integrating Chroma DB for data storage, FAISS for vector search, and Llama 2 via Ollama for response generation, it can deliver precise, context-aware answers to user queries. Each part of the code is structured for clarity, following best practices to ensure a seamless experience, from setup to deployment. Let's dive in and bring this RAG app to life!








In [ ]:
%pip install faiss
%pip install langchain_ollama
%pip install faiss-cpu
%pip install --upgrade gradio


   ---------------------------------------- 14.9/14.9 MB 808.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install arxiv
%pip install pymupdf

   ---------------------------------------- 16.2/16.2 MB 557.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:

import langchain
import langchain.vectorstores
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.vectorstores import faiss
from langchain_community.vectorstores import FAISS

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader, PyPDFLoader
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [17]:
import gradio as gr
from functools import partial
from operator import itemgetter
import json
from pprint import pprint

In [18]:
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
    )

In [19]:
#import Llama2 
instruct_llm = ChatOllama(model="llama2", temperature=0.6, num_predict=256)

#Using Nvidia embeddings
embedder= OllamaEmbeddings(model="llama2")

In [20]:
docs=[
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    #ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    #ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    #ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    #ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    
    ]


In [21]:
for doc in docs:
    content=json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

In [22]:
#Chunking the documents and remove very short chunks
print("Start chunking")
doc_chunks=[text_splitter.split_documents(doc) for doc in docs]
doc_chunks=[[c for c in dchunks if len(c.page_content)>200] for dchunks in doc_chunks]

Start chunking


In [23]:
#Adding the big-picture details
Doc_string="Available Documents: "
Doc_metadata=[]
for chunk in doc_chunks:
    metadata= getattr(chunk[0], 'metadata',{})
    Doc_string+= "\n - " + metadata.get('Title')
    Doc_metadata+= [str(metadata)]
    
BP_Chunks=  [Doc_string] + Doc_metadata

In [24]:
## Printing out some summary information for reference
print(Doc_string, '\n')
for i, chunks in enumerate(doc_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Available Documents: 
 - Attention Is All You Need
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena 

Document 0
 - # Chunks: 35
 - Metadata: 
{'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion '
            'Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin',
 'Published': '2023-08-02',
 'Summary': 'The dominant sequence transduction models are based on complex '
            'recurrent or\n'
            'convolutional neural networks in an encoder-decoder '
            'configuration. The best\n'
            'performing models also connect the encoder and decoder through an '
            'attention\n'
            'mechanism. We propose a new simple network architecture, the '
            'Transformer, based\n'
            'solely on attention mechanisms, dispensing with recurrence and '
            'convolutions\n'
            'entirely. Experiments on two machine translation tasks show these '
            'models to be\n'
            'superio

In [25]:
#Consctructing the vector store
vecstore=[FAISS.from_texts(BP_Chunks, embedding=embedder)]
vecstore+=[FAISS.from_documents(doc_chunk,embedding=embedder) for  doc_chunk in doc_chunks]  


In [26]:
embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

In [27]:
def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity.
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

In [28]:
## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstore)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 82 chunks


In [29]:
convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

In [ ]:
initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {Doc_string}\n\nHow can I help you?"
)


chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
    "Be concize and precize to answer in less than 250 words"
), ('user', '{input}')])

In [31]:
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))



In [32]:
stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

In [33]:
## Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

retrieval_chain = (
    {'input' : (lambda x: x)}
    | RunnableAssign({'history' : itemgetter('input')| convstore.as_retriever()| long_reorder | docs2str })
    | RunnableAssign({'context' : itemgetter('input')|  docstore.as_retriever()| long_reorder | docs2str})
)

In [34]:
def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)



In [35]:
## Start of Agent Event Loop
test_question = "Tell me about RAG!"  

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

ChatPromptValue(messages=[SystemMessage(content='You are a document chatbot. Help the user as they ask questions about documents. User messaged just asked: Tell me about RAG!\n\n From this, we have retrieved the following potentially-useful info:  Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from Attention Is All You Need] .1, instead of 0.3.\\nFor the base models, we used a single model obtained by averaging the last 5 checkpoints, which\\nwere written at 10-minute intervals. For the big models, we averaged the last 20 checkpoints. We\\nused beam search with a beam size of 4 and length penalty \\u03b1 = 0.6 [38]. These hyperparameters\\nwere chosen after experimentation on the development set. We set the maximum output length during\\ninference to input length + 50, but terminate early when possible [38].\\nTable 2 summarizes our results and compares our translation quality and training costs to other model\\narchitectures from the literature. We estimate the numb

In [36]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    

    demo.launch(debug=True, share=True, show_api=False)
    
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e


c:\Users\MSI\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c13aa2db04d38efd90.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ChatPromptValue(messages=[SystemMessage(content='You are a document chatbot. Help the user as they ask questions about documents. User messaged just asked: What is the core idea behind the LLM-a-Judge concept?\n\n From this, we have retrieved the following potentially-useful info:  Conversation History Retrieval:\n[Quote from Document] User previously responded with Tell me about RAG!\n[Quote from Document] Agent previously responded with Ah, another exciting topic related to natural language processing! RAG, or the Recent Advances in Generative models, is a fascinating area of research that has seen significant developments in recent years.\n\nRAG refers to the latest advancements in generative models, such as transformer-based architectures like BERT and RoBERTa, which have shown remarkable performance in various natural language processing tasks. These models are capable of generating coherent and contextually relevant text, thanks to their ability to learn from large amounts of dat

In [51]:
## Save and compress your index
docstore.save_local("Mydocstore_index")

In [52]:
!tar czvf Mydocstore_index.tgz Mydocstore_index

!rm -rf Mydocstore_index



a Mydocstore_index
a Mydocstore_index/index.faiss
a Mydocstore_index/index.pkl
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [53]:
#Make sure the retreival from Mydocstore_index works
!tar xzvf Mydocstore_index.tgz
new_db = FAISS.load_local("Mydocstore_index", embedder, allow_dangerous_deserialization=True)
docs_test = new_db.similarity_search("Testing the index")
print(docs_test[0].page_content[:1000])

x Mydocstore_index/
x Mydocstore_index/index.faiss
x Mydocstore_index/index.pkl


. Our results indicate that using LLM-as-a-judge to approximate\nhuman preferences is highly feasible and could become a new standard in future benchmarks. We\nare also hosting a regularly updated leaderboard with more models 2. Notably, DynaBench [21], a\nresearch platform dedicated to dynamic data collection and benchmarking, aligns with our spirit.\nDynaBench addresses the challenges posed by static standardized benchmarks, such as saturation and\noverfitting, by emphasizing dynamic data with human-in-the-loop. Our LLM-as-a-judge approach\ncan automate and scale platforms of this nature.\n6\nDiscussion\nLimitations. This paper emphasizes helpfulness but largely neglects safety. Honesty and harm-\nlessness are crucial for a chat assistant as well [2]. We anticipate similar methods can be used to\nevaluate these metrics by modifying the default prompt


**Now our DocQuery pipeline works fine and stores locally the files successfully. In the next steps, we will use ChromaDB for effecient vector store and retreival. Furthermore, we will try to implement the concept of LLM-as-Judge to evaluate the results generated by our model.**